<a href="https://colab.research.google.com/github/jdgilhuly/kelly-criterion/blob/main/kelly_Criterion_Option_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Kelly Criterion/ Option Moddeling**

In [1]:
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
import numpy as np
import pandas as pd
from pandas import DataFrame

#Kelly Criterion

Kelly criterion, is a formula that determines the optimal theoretical size for a bet. It is valid when the expected returns are known. The Kelly bet size is found by maximizing the expected value of the logarithm of wealth, which is equivalent to maximizing the expected geometric growth rate.

##Gambling Formula Kelly (full losses)


Where losing the bet involves losing the entire wager, the Kelly bet is:

$f^* = p - \frac{q}{b} = p + \frac{p-1}{b}$

where:

$f^*$ is the fraction of the current bankroll to wager.

$p$ is the probability of a win.

$q$ is the probability of a loss ($q = 1-p$)

$b$ is the proportion of the bet gained with a win. E.g. If betting 10 on a 2-to-1 odds bet, then $b = 20/10 = 2.0$



In [27]:
def gambling_kelly(prob_of_win,proportion_bet_win):
  fraction = prob_of_win-(prob_of_win/proportion_bet_win)
  return fraction

##Investment formula Kelly (partial losses)
A more general form for partial losses (e.g. investing):

$f^* = \frac{p}{a} - \frac{q}{b}$

where:

$f^*$ is the fraction of the assets to apply to the security.

$p$ is the probability that the investment increases in value  

$q$ is the probability that the investment decreases in value  ($q = 1-p$)

$a$ is the fraction that is lost in a negative outcome. E.g. If the security price falls 10%, then $a=0.1$

$b$ is the fraction that is gained in a positive outcome. E.g. If the security price rises 10%, then $b=0.1.$


Note that the Kelly Criterion is only valid for known outcome probabilities, which is not the case with investments. It is not recommended that risk averse investors invest the full Kelly fraction.

This formula can result in Kelly fractions higher than 1. In this case, it is theoretically advantageous to use leverage to purchase additional securities on margin.

In [40]:
def investment_kelly(prob_of_win,win_fraction,lost_fraction):
    fraction = (prob_of_win/lost_fraction)- ((1-prob_of_win)/win_fraction)
    return fraction

##Some Examples

In [36]:
def gambling_wager_amount(prob_of_success,bankroll,prop_bet_gained):
      print('bankroll of {} * optimal fraction to wager {} = {}'.format(bankroll,gambling_kelly(prob_of_success,prop_bet_gained), bankroll * gambling_kelly(prob_of_success,prop_bet_gained)))
      return bankroll * gambling_kelly(prob_of_success,2)

In [38]:
gambling_wager_amount(0.6,1000,3)

bankroll of 1000 * optimal fraction to wager 0.4 = 400.0


300.0

In [41]:
def investing_wager_amount(prob_of_success,bankroll,win_fraction, loss_fraction):
      print('bankroll of {} * optimal fraction to wager {} = {}'.format(bankroll,investment_kelly(prob_of_success,win_fraction,loss_fraction), investment_kelly(prob_of_success,win_fraction,loss_fraction)))
      return bankroll * investment_kelly(prob_of_success,win_fraction,loss_fraction)

In [42]:
investing_wager_amount(0.6,1000,0.2,0.1)

bankroll of 1000 * optimal fraction to wager 3.999999999999999 = 3.999999999999999


3999.999999999999

#Option Modeling with Black-Scholes

The Black–Scholes is a mathematical model for the dynamics of a financial market containing derivative investment instruments. The Black–Scholes formula gives a theoretical estimate of the price of European-style options and shows that the option has a unique price given the risk of the security and its expected return.

The key idea behind the model is to hedge the option by buying and selling the underlying asset in just the right way and, as a consequence, to eliminate risk. This type of hedging is called "continuously revised delta hedging" and is the basis of more complicated hedging strategies such as those engaged in by investment banks and hedge funds.

##Assumptions:

**Now assumptions are made on the assets (which explain their names):**

Riskless rate: The rate of return on the riskless asset is constant and thus called the risk-free interest rate.

Random walk: The instantaneous log return of stock price is an infinitesimal random walk with drift; more precisely, the stock price follows a geometric Brownian motion, and it is assumed that its drift and volatility are constant (if they are time-varying, a suitably modified Black–Scholes formula can be deduced quite simply, as long as the volatility is not random).

The stock does not pay a dividend.


**The assumptions on the market are:**

No arbitrage opportunity (i.e., there is no way to make a riskless profit).

Ability to borrow and lend any amount, even fractional, of cash at the riskless rate.

Ability to buy and sell any amount, even fractional, of the stock (this includes short selling).

The above transactions do not incur any fees or costs (i.e., frictionless market).

##American Option

Since the American option can be exercised at any time before the expiration date, the Black–Scholes equation becomes a variational inequality of the form:

$\frac {∂V}{∂t} + \frac {1}{2}σ^2 S^2\frac {∂^2V}{∂S^2}+rS \frac {∂V}{∂S}- rV \leq 0$

together with $V(S,t)\geq H(S)$ where $H(S)$ denotes the payoff at stock price $S$ and the terminal condition: $V(S,T)=H(S)$.

Where:

General and market related:

$t$ is a time in years; with $t=0$ generally representing the present year.

$r$ is the annualized risk-free interest rate, continuously compounded (also known as the force of interest).

Asset related:

$S(t)$ is the price of the underlying asset at time t, also denoted as $S_t$.

$\mu$  is the drift rate of $S$, annualized.

$ \sigma $  is the standard deviation of the stock's returns. This is the square root of the quadratic variation of the stock's log price process, a measure of its volatility.

Option related:

$V(S,t)$ is the price of the option as a function of the underlying asset S at time t, in particular:

$C(S,t)$ is the price of a European call option

$P(S,t)$ is the price of a European put option.

$T$ is the time of option expiration.
$\tau$  is the time until maturity: $\tau =T-t$.

$K$ is the strike price of the option, also known as the exercise price.

In [2]:
def d1(S,K,T,r,sigma):
    return(log(S/K)+(r+sigma**2/2.)*T)/(sigma*sqrt(T))
    
def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma)-sigma*sqrt(T)

def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma))-K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))
  
def bs_put(S,K,T,r,sigma):
    return K*exp(-r*T)-S+bs_call(S,K,T,r,sigma)


In [ ]:
def call_implied_volatility(Price, S, K, T, r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = S * \
            norm.cdf(d1(S, K, T, r, sigma))-K*exp(-r*T) * \
            norm.cdf(d2(S, K, T, r, sigma))
        if Price-(Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"

def put_implied_volatility(Price, S, K, T, r):
    sigma = 0.001
    while sigma < 1:
        Price_implied = K*exp(-r*T)-S+bs_call(S, K, T, r, sigma)
        if Price-(Price_implied) < 0.001:
            return sigma
        sigma += 0.001
    return "Not Found"


In [ ]:
def call_delta(S,K,T,r,sigma):
    return norm.cdf(d1(S,K,T,r,sigma))
def call_gamma(S,K,T,r,sigma):
    return norm.pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))
def call_vega(S,K,T,r,sigma):
    return 0.01*(S*norm.pdf(d1(S,K,T,r,sigma))*sqrt(T))
def call_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm.pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) - r*K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma)))
def call_rho(S,K,T,r,sigma):
    return 0.01*(K*T*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma)))
    
def put_delta(S,K,T,r,sigma):
    return -norm.cdf(-d1(S,K,T,r,sigma))
def put_gamma(S,K,T,r,sigma):
    return norm.pdf(d1(S,K,T,r,sigma))/(S*sigma*sqrt(T))
def put_vega(S,K,T,r,sigma):
    return 0.01*(S*norm.pdf(d1(S,K,T,r,sigma))*sqrt(T))
def put_theta(S,K,T,r,sigma):
    return 0.01*(-(S*norm.pdf(d1(S,K,T,r,sigma))*sigma)/(2*sqrt(T)) + r*K*exp(-r*T)*norm.cdf(-d2(S,K,T,r,sigma)))
def put_rho(S,K,T,r,sigma):
    return 0.01*(-K*T*exp(-r*T)*norm.cdf(-d2(S,K,T,r,sigma)))

In [ ]:
call_delta(150,160,20,0.02,0.3491)

0.8402491338582363